In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import sqlite3
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

From databases created with sqlite, queried multiple tables to isolate Vanderbilt facilities that fell within the following parameters: CBSA 34980, to_npi(Vanderbilt facilitites that were referred to) and entity_type_code 2 (hospitals)   

In [ ]:
query = '''
WITH hospital_npi_data AS(
    SELECT 
        DISTINCT entity_type_code AS entity,
        npi,
        `provider_organization_name_(legal_business_name)` AS organization,
        SUBSTR(provider_business_practice_location_address_postal_code, 1, 5) AS zipcode,
        provider_business_practice_location_address_state_name AS state,
        taxonomy_code
    FROM 
        npi_data
    WHERE
        entity_type_code = 2
)
    SELECT
        organization,
        to_npi,
        code,
        grouping,
        classification,
        specialization,
        zip,
        state,
        patient_count AS total_patients 
    FROM 
        hop
    INNER JOIN 
        hospital_npi_data 
    ON 
        hop.to_npi = hospital_npi_data.npi
    LEFT JOIN 
        taxonomy
    ON 
        taxonomy_code = code
    LEFT JOIN 
        zipcode
    ON 
        zipcode.zip = hospital_npi_data.zipcode
    WHERE 
        cbsa = 34980
        AND organization in ('VANDERBILT UNIVERSITY MEDICAL CENTER', 
            'MONROE CARELL JR VANDERBILT CHILDREN''S HOSPITAL', 
            'VANDERBILT PSYCHIATRIC HOSPITAL, LLC',
            'VANDERBILT HEART AND VASCULAR INSTITUTE', 
            'VANDERBILT ORTHOPAEDIC INSTITUTE', 
            'VANDERBILT ASTHMA SINUS AND ALLERGY PROGRAM, LLC',
            'VANDERBILT UNIVERSITY MEDICAL CENTER', 
            'VANDERBILT ORTHOPAEDICS AND REHABILITATION COOL SPRINGS', 
            'VANDERBILT COMMUNITY MENTAL HEALTH CENTER', 
            'VANDERBILT COMMUNITY AND HOME SERVICES', 
            'VANDERBILT BEDFORD HOSPITAL, LLC',
            'VANDERBILT COFFEE HOSPITAL, LLC',  
            'VANDERBILT INTEGRATED PROVIDERS, LLC', 
            'VANDERBILT HEALTH AND WILLIAMSON MEDICAL CENTER CLINICS AND SERVICES',
            'VANDERBILT-INGRAM CANCER CENTER AT TENNOVA HEALTHCARE-CLARKSVILLE', 
            'VANDERBILT MAURY RADIATION ONCOLOGY, LLC'
            )
    
'''

In [ ]:
with sqlite3.connect('hop_teaming.sqlite') as db: 
    TO_CBSA = pd.read_sql(query,db)
    
TO_CBSA

After running query, selected 'total_patients' column and summed for all patients within specified parameters of the query.

total patients = 2032854

In [ ]:
TO_CBSA['total_patients'].sum() 

Grouped by 'organization' column and summed 'total_patients' column to get total patient count for each Vanderbilt facility within the parameters specified within query.

Vanderbilt University Medical Center = 2032652  
Vanderbilt Maury Radiation Oncology, LLC = 202

In [ ]:
TO_CBSA.groupby('organization')[('total_patients')].sum().sort_values(ascending = False)  

From databases created with sqlite, using identical parameters as above, queried multiple tables to exclude all Vanderbilt facilities from the dataframe. 

In [ ]:
query = '''
WITH hospital_npi_data AS(
    SELECT 
        DISTINCT entity_type_code AS entity,
        npi,
        `provider_organization_name_(legal_business_name)` AS organization,
        SUBSTR(provider_business_practice_location_address_postal_code, 1, 5) AS zipcode,
        provider_business_practice_location_address_state_name AS state,
        taxonomy_code
    FROM 
        npi_data
    WHERE
        entity_type_code = 2
)
    SELECT
        organization,
        from_npi,
        to_npi,
        code,
        grouping,
        classification,
        specialization,
        zip,
        state,
        patient_count AS total_patients 
    FROM 
        hop
    INNER JOIN 
        hospital_npi_data 
    ON 
        hop.to_npi = hospital_npi_data.npi
    LEFT JOIN 
        taxonomy
    ON 
        taxonomy_code = code
    LEFT JOIN 
        zipcode
    ON 
        zipcode.zip = hospital_npi_data.zipcode
    WHERE 
        cbsa = 34980 
        AND state = 'TN'
        AND organization NOT IN(
            'VANDERBILT UNIVERSITY MEDICAL CENTER', 
            'MONROE CARELL JR VANDERBILT CHILDREN''S HOSPITAL', 
            'VANDERBILT PSYCHIATRIC HOSPITAL, LLC',
            'VANDERBILT HEART AND VASCULAR INSTITUTE', 
            'VANDERBILT ORTHOPAEDIC INSTITUTE', 
            'VANDERBILT ASTHMA SINUS AND ALLERGY PROGRAM, LLC',
            'VANDERBILT UNIVERSITY MEDICAL CENTER', 
            'VANDERBILT ORTHOPAEDICS AND REHABILITATION COOL SPRINGS', 
            'VANDERBILT COMMUNITY MENTAL HEALTH CENTER', 
            'VANDERBILT COMMUNITY AND HOME SERVICES', 
            'VANDERBILT BEDFORD HOSPITAL, LLC',
            'VANDERBILT COFFEE HOSPITAL, LLC',  
            'VANDERBILT INTEGRATED PROVIDERS, LLC', 
            'VANDERBILT HEALTH AND WILLIAMSON MEDICAL CENTER CLINICS AND SERVICES',
            'VANDERBILT-INGRAM CANCER CENTER AT TENNOVA HEALTHCARE-CLARKSVILLE', 
            'VANDERBILT MAURY RADIATION ONCOLOGY, LLC'
            )
    
'''

In [ ]:
with sqlite3.connect('hop_teaming.sqlite') as db: 
    CBSA_NOT_VANDY = pd.read_sql(query,db)
    
CBSA_NOT_VANDY

After running query, selected 'total_patients' column and summed for all patients to all facilitites within specified parameters of the query that are NOT a Vanderbilt facility.

total_patients = 22282295

In [ ]:
CBSA_NOT_VANDY[['total_patients']].sum()

Grouped by 'organization' column and summed 'total_patients' column to get total patient count for each facility within the parameters specified within query that is NOT a Vanderbilt facitlity

In [ ]:
CBSA_NOT_VANDY.groupby('organization')[('total_patients')].sum().sort_values(ascending = False).head(20)

This graph shows the total patient count for the top 20 facilitites within the CBSA 38940  

In [ ]:
c = ['black','black', 'black','black','black','black','black','black','black','black','black','black', 'black','black','black','black','black','black','black','gold']
plt.figure(figsize=(17,10))
cbsa_sorted = cbsa.sort_values(ascending = True)
ax = cbsa_sorted.plot.barh(x='organization', y= 'patients_to', color = c)
plt.xlabel('Number of Patients Referred', fontsize = 12)
plt.ylabel('Organization', fontsize = 12)
plt.xticks(rotation=45, ha = 'right')
plt.legend(['TOTAL PATIENTS: 24,319,802'], loc = 'lower right', bbox_to_anchor=(.5, .1))
plt.title('Top 20 Organizations by Total Patients Referred that fall under the Designations: Entity 2, CBSA 34980')


plt.show()

This graph shows the total patient count for the top 20 facilitites within the city of Nashville, TN.

In [ ]:
c = ['black','black', 'black','black','black','black','black','black','black','black','black','black', 'black','black','black','black','black','black','gold','black']
plt.figure(figsize=(17,10))
ALL_NASH_TOP_20_sorted = ALL_NASH_TOP_20.sort_values(ascending = True)
ax = ALL_NASH_TOP_20_sorted.plot.barh(x='organization', y= 'patients_to', color = c)
plt.xlabel('Number of Patients Referred', fontsize = 12)
plt.ylabel('Organization', fontsize = 12)
plt.xticks(rotation=45, ha = 'right')
plt.legend(['TOTAL PATIENTS: 10,657,999'], loc = 'lower right', bbox_to_anchor=(.5, .1))
plt.title('Top 20 Organizations by Total Patients Referred that fall under the Designations: Entity 2, Nashville, TN')


plt.show()

This graph shows the percentage of patient share for Vanderbilt when compared to all other healthcare facilities in Nashville. 

In [ ]:
total_patients = 10657999
vandy_percent = 11
not_vandy_percent = 89

# Create a vertical bar plot
fig, ax = plt.subplots(figsize=(11,8))
ax.bar(['Vanderbilt', 'Others'], [vandy_percent, not_vandy_percent], color=['gold', 'black'])
ax.set_xlabel('Percentage of Total Patients', fontsize = 14)
ax.set_ylim(0, 100)
ax.set_title('Vanderbilt vs All Others'.format(total_patients))

# Add the percentages to the bars
for i, v in enumerate([vandy_percent, not_vandy_percent]):
    ax.text(i, v + 1, '{}%'.format(v), color='black',fontsize = 14, fontweight='bold', ha='center')

plt.show()